In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
#Metadaten einlesen
df = pd.read_csv("NIHMS841832-supplement-1.csv", sep=',')
    
#Ergebnisse des Feature Tables einlesen
feature = pd.read_csv('feature_table.txt', sep='\t').T
feature = feature[1:][:-1]

feature

,0,1,2,3,4,5,6,7,8,9,...,323,324,325,326,327,328,329,330,331,332
1629.SubjectIBD335,0,345,0,0,0,0,412,0,0,0,...,0,0,0,0,0,0,7,0,0,353
1629.SubjectIBD643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1629.SubjectIBD539,0,2869,0,0,0,0,1665,0,0,0,...,0,746,0,0,0,3,21,0,0,88919
1629.SubjectIBD078,0,5,0,0,0,0,17,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1629.SubjectIBD671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629.SubjectIBD238,0,0,0,0,0,0,50,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1629.SubjectIBD421,0,560,0,98,0,0,2368,0,0,0,...,0,68,0,0,6,22,259,0,650,2578
1629.SubjectIBD202,0,0,0,0,0,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,31
1629.SubjectIBD544,0,0,0,0,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,464937


In [3]:
#Gesunde Kontrollgruppe
HC = df[df.ibd_subtype.eq("HC")]

y = []
for row in feature.index:
    if any(True for val in HC['sample_name'] if val == row):
        y.append(1)
    else:
        y.append(0)

X = feature.iloc[:, :].values  

In [7]:
#Split Test und Trainingsdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [8]:
#Featuer Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Train
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

#Evaluation Train Data
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

#Evaluating Test Data
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[463   0]
 [  4  43]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       463
           1       1.00      0.91      0.96        47

    accuracy                           0.99       510
   macro avg       1.00      0.96      0.98       510
weighted avg       0.99      0.99      0.99       510

[[151   5]
 [  7   7]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       156
           1       0.58      0.50      0.54        14

    accuracy                           0.93       170
   macro avg       0.77      0.73      0.75       170
weighted avg       0.93      0.93      0.93       170



In [6]:
f1 = 0.0
n = 100
for i in range(0, n):
    #Split Test und Trainingsdaten
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = n)

    #Featuer Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    #Train
    mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
    mlp.fit(X_train,y_train)

    predict_train = mlp.predict(X_train)
    predict_test = mlp.predict(X_test)
    
    #Test
    y_pred = mlp.predict(X_test)
    predict_train = mlp.predict(X_train)
    f1 += f1_score(y_test, y_pred, average='macro')
f1 = f1/n
print("f1-score = "+str(f1))

f1-score = 0.8223077459986805
